In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm, trange

import dtt_fiber_processing

pd.options.mode.chained_assignment = None

In [4]:
data_dir = Path('R:\\3_Histology\\Qupath Projects\\export measurements Test65\\')
query_file, data_files = dtt_fiber_processing.parse_datafile_paths(data_dir)
#data_file = data_files[0] # We're going to work with one file for now

data_file = data_dir.joinpath('24_3_ACP.csv')
data = pd.read_csv(data_file)


In [7]:
tree, id_map, structure_map = dtt_fiber_processing.get_atlas_components()

In [9]:
left_data, right_data = dtt_fiber_processing.split_data(data)

left_data.reset_index(inplace=True, drop=True)
right_data.reset_index(inplace=True, drop=True)

fiber_key = [ids for ids in id_map if 'fiber' in ids][0]
id_map['fiber_tracts'] = id_map[fiber_key]

In [10]:
left_acronyms = left_data['Classification']
right_acronyms = right_data['Classification']
left_ids = [id_map[region] for region in left_acronyms]
right_ids = [id_map[region] for region in right_acronyms]

In [12]:
left_data.insert(1, 'ID', left_ids)
right_data.insert(1, 'ID', right_ids)

In [14]:
left_data['Artifact Size'] = left_data['Artifact Size'].fillna(0)
right_data['Artifact Size'] = right_data['Artifact Size'].fillna(0)

In [16]:
left_paths = []
for id in left_ids:
    path = structure_map[id]
    left_paths.append(path)

right_paths = []
for id in right_ids:
    path = structure_map[id][1:]
    right_paths.append(path)

In [18]:
left_data.insert(len(left_data.columns), 'Path', left_paths)
right_data.insert(len(right_data.columns), 'Path', right_paths)